Hi,
so this is the first thing that comes to my mind:

1. $n$ agents, $m$ projects
1. in iterations, add "chaos":
  1. randomly choose a project $p$ and a random cycle $C$ on a set of some $z$ agents that are not yet doing anything with this project $p$ **and at least one other project** (can be different for each voter in $C$);
  1. for each agent in this cycle, choose a superset of projects $S$, $p \in S$, to delegate to the next agent on the cycle (e.g., if we choose project $p_1$ and agents $a, b, c$, we can delegate $\{p_1,p_2\}$ from $a$ to $b$; delegate $\{p_1, p_3, p_{65}\}$ from $b$ to $c$; and delegate $\{p_1, p_6\}$ from $c$ to $a$)
  1. halt whenever you cannot find more chaos to add = there is no project $p$ which is non-delegated by $\geq 2$ agents s.t. each agent has at least one other non-delegated project.
1. the variables are budgets for the parts of the partition




In [1]:
import pyomo.environ as pyo
from instance_generator import create
import os

The result of the `create` function is a dictionary `res` indexed by the voters, such that for a voter $v$, `res[v]` is the partition of projects, e.g. `res[v][S]=d` if $v$ delegates $S \subseteq [m]$ to $d \in [n]$, and if $d=v$ then it means that $v$ is delegating this to himself. 

The optimization problem is to find an two fixed points $x_1, x_2$ which are as far away from each other as possible.

## The Model

In [2]:
def uniqueness_counterexample_model(res, n,m,fix_weights=False, even_split=False):
    model = pyo.ConcreteModel()

    model.two = pyo.RangeSet(1,2)
    model.I = pyo.RangeSet(0, n-1)
    model.J = pyo.RangeSet(0, m-1)

    model.x = pyo.Var(model.two, model.I, model.J, domain=pyo.UnitInterval)
        # We have x[1] and what it to satisfy proportionality with itself
        # x[2] is x, which we will want to violate pseudo-monotonicity
        # x[3] is f(x[2]) = f(x) because those are the coefficients in the pseudogradient...
    model.d = pyo.Var(model.I, model.J, domain=pyo.UnitInterval) # defaults

    model.D = pyo.Set(initialize=list((v,S) for v in res for S in res[v])) # delegated sets
    model.DD = pyo.Set(initialize=list(((v,S,p) for v in res for S in res[v] for p in S))) # delegated sets \times projects
    model.DDD = pyo.Set(initialize=list(((i,v,S,p) for i in model.two for v in res for S in res[v] for p in S)))
    model.b = pyo.Var(model.D, domain=pyo.UnitInterval) # budget variables
    
    if not fix_weights:
        model.w = pyo.Var(model.D, domain=pyo.NonNegativeReals, bounds=(1,10)) # delegation weights variables
        #model.w = pyo.Var(model.D, domain=pyo.Integers, bounds=(1,5)) # delegation weights variables
    else:
        model.w = {(v,S): 10 for v in res for S in res[v]}
    
    def x_budget_constraint_rule(m,it,v,S):
        return sum(model.x[(it,v,j)] for j in S) == model.b[(v,S)]

    model.x_budget_constraints = pyo.Constraint(model.two, model.D, rule=x_budget_constraint_rule)

    def defaults_constraint_rule(m,v,S):
        return sum(model.d[(v,j)] for j in S) == model.b[(v,S)]
    
    def defaults_constraint_even_split_rule(m,v,S,p):
        return model.d[(v,p)] == model.b[(v,S)] / len(S)

    if even_split:
        model.defaults_constraints = pyo.Constraint(model.DD, rule=defaults_constraint_even_split_rule)
    else:
        model.defaults_constraints = pyo.Constraint(model.D, rule=defaults_constraint_rule)

    def budget_simplex_constraint_rule(m,v):
        return sum(model.b[(v,S)] for S in res[v]) == 1

    model.budget_simplex_constraints = pyo.Constraint(model.I, rule=budget_simplex_constraint_rule)
    # model.b[(v,S)] = how much budget we give delegation S
    # model.w[(v,S)] = how much weight we give the delegate


    # m is the model, always an implicit argument
    # SHOULD BE IMPLIED by budget_simplex_constraint_rule
    # def simplex_rule(m, i):
    #    return sum(m.x[(1,i,j)] for j in m.J) == 1

    # the next line creates one constraint for each member of the set model.I
    # sum of contributions of one voter over all projects is 1
    # model.simplex_constraints = pyo.Constraint(model.I, rule=simplex_rule)
    # Proportionality constraints

    def proportionality_rule(m,it1, it2,v,S,p):
        # m is model, v voter, S delegated subset, it=1 or 2 is iteration
        delegate = res[v][S]
        denominator = sum(m.d[(v,p)] + m.w[(v,S)]*m.x[(it1,delegate,p)] for p in S)
        return m.x[(it2,v,p)] == ((m.d[(v,p)] + m.w[(v,S)]*m.x[(it1,delegate,p)]) / denominator) * m.b[(v,S)]
    
    def self_prop_rule(m,v,S,p):
        return proportionality_rule(m,1,1,v,S,p)
    
    def second_selfprop_rule(m,v,S,p):
        return proportionality_rule(m,2,2,v,S,p)
    
    model.self_proportionality = pyo.Constraint(model.DD, rule=self_prop_rule)
    model.second_selfprop_proportionality = pyo.Constraint(model.DD, rule=second_selfprop_rule)
    
    def objective_rule(m):
        obj = sum(abs(m.x[(1,i,j)] - m.x[(2,i,j)]) for i in m.I for j in m.J)
        return obj
    model.OBJ = pyo.Objective(rule=objective_rule,sense=pyo.maximize)
    
    return model

In [22]:
n=10
m=5
res = create(n,m)

In [23]:
for i in range(1000):
    res = create(n,m)
    model = uniqueness_counterexample_model(res, n,m,fix_weights=True, even_split=False)
    #os.environ["NEOS_EMAIL"] = "koutecky@iuuk.mff.cuni.cz"
    #opt = pyo.SolverFactory("knitro")
    #solver_manager = pyo.SolverManagerFactory('neos')
    opt = pyo.SolverFactory("ipopt")
    opt.options['max_iter']= 100000
    # solver_manager = pyo.SolverManagerFactory('neos')
    # results = solver_manager.solve(model, opt=opt,tee=True)
    %time results = opt.solve(model)
    try:
        obj = sum(abs(model.x[(1,i,j)].value - model.x[(2,i,j)].value) for i in model.I for j in model.J)
    except TypeError:
        continue
    print()
    print("*************************")
    print("obj:", obj)
    print("*************************")
    print()
    if obj > 0.1:
        print()
        print("************ FOUND IT !!! ************************")
        break

    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.14.4\x3a Maximum Number of Iterations
        Exceeded.
CPU times: user 279 ms, sys: 84.2 ms, total: 363 ms
Wall time: 4.37 s

*************************
obj: 7.524841138389915e-06
*************************

    model.name="unknown";
      - termination condition: other
      - message from solver: Too few degrees of freedom (rethrown)!
CPU times: user 26.9 ms, sys: 8.58 ms, total: 35.5 ms
Wall time: 44.7 ms
    model.name="unknown";
      - termination condition: other
      - message from solver: Too few degrees of freedom (rethrown)!
CPU times: user 39.9 ms, sys: 0 ns, total: 39.9 ms
Wall time: 47.5 ms
CPU times: user 35.1 ms, sys: 6.76 ms, total: 41.9 ms
Wall time: 85.7 ms

*************************
obj: 6.318759404599916e-10
*************************

    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.14.4\x3a Maximum N

In [24]:
# Weights are all 10
# Default are all evensplit
import numpy as np
b = dict()
for v in res:
    for S in res[v]:
        b[(v,S)] = model.b[(v,S)].value
# x is fixedpoint, y is some solution
x1 = np.zeros((n,m))
x2 = np.zeros((n,m))
for i in model.I:
    for j in model.J:
        x1[i,j] = model.x[(1,i,j)].value
        x2[i,j] = model.x[(2,i,j)].value
counterexample = {"res": res, "x1": x1, "x2": x2, "b":b}

In [25]:
#import cloudpickle as pickle
import pickle
name="n_10_m_5_nonuniqueness_fixedweight"
backup = open(name+".pickle","wb")
#model_file = open(name+"_model.pickle","wb")
pickle.dump(counterexample,backup)
backup.close()

In [26]:
res

defaultdict(dict,
            {5: {frozenset({1, 2, 3}): 8,
              frozenset({0}): 5,
              frozenset({4}): 5},
             8: {frozenset({0, 1, 2, 3, 4}): 2},
             2: {frozenset({1, 2}): 3,
              frozenset({3, 4}): 7,
              frozenset({0}): 2},
             3: {frozenset({2, 4}): 1, frozenset({0, 1, 3}): 7},
             1: {frozenset({0, 1, 2, 4}): 0, frozenset({3}): 1},
             0: {frozenset({0, 1, 2, 3, 4}): 9},
             9: {frozenset({0, 1, 2, 3}): 6, frozenset({4}): 9},
             6: {frozenset({0, 1, 2, 3, 4}): 4},
             4: {frozenset({0, 1, 2, 3}): 5, frozenset({4}): 4},
             7: {frozenset({0, 1, 4}): 3, frozenset({2, 3}): 2}})

In [27]:
b

{(5, frozenset({1, 2, 3})): 0.999999987471292,
 (5, frozenset({0})): -8.252325947214718e-09,
 (5, frozenset({4})): 2.078103380685064e-08,
 (8, frozenset({0, 1, 2, 3, 4})): 1.0,
 (2, frozenset({1, 2})): 0.26591629226527796,
 (2, frozenset({3, 4})): 0.7340837151136113,
 (2, frozenset({0})): -7.378889127230838e-09,
 (3, frozenset({2, 4})): -8.86594159605404e-09,
 (3, frozenset({0, 1, 3})): 1.000000008865944,
 (1, frozenset({0, 1, 2, 4})): 0.9999999994066154,
 (1, frozenset({3})): 5.933846042139484e-10,
 (0, frozenset({0, 1, 2, 3, 4})): 1.0,
 (9, frozenset({0, 1, 2, 3})): 1.0000000056573894,
 (9, frozenset({4})): -5.657389342835202e-09,
 (6, frozenset({0, 1, 2, 3, 4})): 1.0,
 (4, frozenset({0, 1, 2, 3})): 1.0000000054631963,
 (4, frozenset({4})): -5.4631962854886245e-09,
 (7, frozenset({0, 1, 4})): 0.4374867054130049,
 (7, frozenset({2, 3})): 0.5625132945869952}

In [28]:
x1

array([[ 2.96885367e-08,  1.91048425e-01,  9.71691101e-02,
         7.11782432e-01,  3.24649803e-09],
       [ 1.15492104e-01,  5.74977310e-01,  2.50295499e-01,
         5.93384604e-10,  5.92350867e-02],
       [-7.37888913e-09,  2.49180567e-01,  1.67357251e-02,
         5.95180294e-01,  1.38903421e-01],
       [-8.32960873e-09,  3.95926265e-01, -7.16926000e-09,
         6.04073753e-01, -1.69668154e-09],
       [ 2.28194755e-08,  2.33270116e-01,  1.18643484e-01,
         6.48086383e-01, -5.46319629e-09],
       [-8.25232472e-09,  2.56597103e-01,  1.30507830e-01,
         6.12895054e-01,  2.07810338e-08],
       [ 2.11779396e-08,  2.12063742e-01,  1.07857713e-01,
         5.89169440e-01,  9.09090848e-02],
       [-9.22748886e-09,  3.93955777e-01,  6.14459629e-02,
         5.01067332e-01,  4.35309379e-02],
       [ 5.89224289e-02,  2.26527788e-01,  1.52142948e-02,
         5.41072994e-01,  1.58262494e-01],
       [ 2.83723120e-08,  2.10153264e-01,  1.06886021e-01,
         6.82960692e-01

In [29]:
x2

array([[ 2.96885353e-08,  1.57532605e-08,  7.44546220e-01,
         2.55453731e-01,  3.24649808e-09],
       [ 5.30889369e-02,  3.80889538e-02,  8.81593167e-01,
         5.93384604e-10,  2.72289419e-02],
       [-7.37888913e-09, -9.78450945e-09,  2.65916302e-01,
        -9.81494118e-09,  7.34083725e-01],
       [ 8.05809429e-01, -9.83043354e-09, -8.60031129e-09,
         1.94190590e-01, -2.65630035e-10],
       [ 2.28194738e-08,  7.48215228e-09,  9.09090936e-01,
         9.09090388e-02, -5.46319629e-09],
       [-8.25232651e-09, -9.41789351e-09,  1.00000001e+00,
        -9.29098922e-09,  2.07810338e-08],
       [ 2.11779381e-08,  7.19042327e-09,  8.26446305e-01,
         8.26445814e-02,  9.09090848e-02],
       [ 4.14957983e-01, -4.38013813e-09,  5.62513304e-01,
        -9.75678294e-09,  2.25287268e-02],
       [ 5.89224289e-02, -9.46569711e-09,  2.41742092e-01,
        -9.48500006e-09,  6.99335498e-01],
       [ 2.83723105e-08,  1.33216566e-08,  8.19000842e-01,
         1.80999122e-01